# Implementación N-gramas

In [1]:
import argparse
import glob
import json
import re
import html
import random
import pickle
import gc
from pathlib import Path
from lxml import etree
from collections import Counter
from typing import Dict, List, Tuple
import nltk

Se definen los PATHS a los datos. En caso de necesitarlo aquí es donde se modifican para poder utilizar los datos locales.

In [2]:
PATH_TO_BAC = "data/BAC/blogs"  # Cambia esta ruta por la de tu carpeta con archivos XML
OUTPUT_BAC_JSONL = "data/processed/BAC.jsonl"  # Ruta del archivo JSONL de salida
PATH_TO_20NG = "data/20news-18828/"  # Cambia esta ruta por la de tu carpeta con archivos de 20 Newsgroups
OUTPUT_20NG_JSONL = "data/processed/20news.jsonl"

OUTPUT_20NG_TOKENIZED_JSONL = "data/processed/20news_tokenized.jsonl"
OUTPUT_BAC_TOKENIZED_JSONL = "data/processed/BAC_tokenized.jsonl"


GROUP_ID = "0100"
OUTPUT_20NG_SPLITS_JSONL = f"splits/20N_{GROUP_ID}_training.jsonl"
OUTPUT_BAC_SPLITS_JSONL = f"splits/BAC_{GROUP_ID}_training.jsonl"

## Carga de datos

Para ambos archivos se decidió que la mejor manera de guardar la información era en formato JSONL, de manera que cada línea del archivo es un JSON independiente. 

### Dataset BAC

Para realizar la unificación de los datos es necesario retirar todas aquellas entidades que generan problemas y que no deberían existir en un archivo MXL. Adicionalmente se retiran todos los caracteres de control que no sean:

- Tab (`\t`)
- Newline (`\n`)
- Carriage return (`\r`)
- Caracteres Unicode válidos (acentos,emojis, etc.)

In [3]:

# Regex: reemplaza & que NO inician entidad válida (&word;, &#123;, &#x1F;)
_AMP_FIX = re.compile(r'&(?!#\d+;|#x[0-9a-fA-F]+;|\w+;)')

# XML 1.0 chars válidos (excluimos controles)
# Permitimos: \x09, \x0A, \x0D, \x20-\uD7FF, \uE000-\uFFFD, \U00010000-\U0010FFFF
_INVALID_XML_CHARS = re.compile(
    r'([^\x09\x0A\x0D\x20-\uD7FF\uE000-\uFFFD'
    r'\U00010000-\U0010FFFF])',
    flags=re.UNICODE
)


In [4]:
def clean_xml_text(raw: str) -> str:
    """
    Limpia texto para que sea más amigable al parser XML aplicando múltiples transformaciones.
    
    Realiza una limpieza en tres pasos:
    1. Decodifica entidades HTML comunes (&nbsp;, &hearts;, etc.) a caracteres Unicode
    2. Escapa ampersands sueltos (& -> &amp;) que no sean entidades válidas
    3. Remueve caracteres inválidos para XML 1.0 según especificación
    
    Args:
        raw (str): Texto crudo que puede contener entidades HTML, ampersands sueltos 
                   y caracteres inválidos para XML
    
    Returns:
        str: Texto limpio y válido para parsing XML, con entidades decodificadas,
             ampersands escapados correctamente y caracteres inválidos removidos
    """

    unescaped = html.unescape(raw)

    amp_fixed = _AMP_FIX.sub("&amp;", unescaped)

    cleaned = _INVALID_XML_CHARS.sub("", amp_fixed)

    return cleaned

def extract_texts(root: etree._Element) -> Tuple[List[str], List[str]]:
    """
    Extrae listas de fechas y posts de un elemento XML, tolerando namespaces.
    
    Utiliza XPath con local-name() para buscar elementos independientemente 
    de sus namespaces.
    
    Args:
        root (etree._Element): Elemento raíz del árbol XML desde donde extraer
                               los elementos 'date' y 'post'
    
    Returns:
        Tuple[List[str], List[str]]: Tupla conteniendo:
            - Lista de strings con fechas encontradas (elementos con local-name='date')
            - Lista de strings con posts encontrados (elementos con local-name='post')
            Ambas listas tienen texto limpio (strip aplicado) y valores vacíos 
            para elementos sin contenido textual
    
    """
    # Selecciona cualquier elemento cuyo local-name sea 'date' o 'post'
    dates = [ (el.text or "").strip()
              for el in root.xpath(".//*[local-name()='date']") ]
    posts = [ (el.text or "").strip()
              for el in root.xpath(".//*[local-name()='post']") ]
    return dates, posts

def xml_to_jsonl(input_dir: str, output_path: str) -> None:
    """
    Convierte todos los archivos XML 'sucios' de un directorio a un único archivo JSONL.
    
    Procesa archivos XML potencialmente malformados o con caracteres inválidos,
    aplicando limpieza automática y parsing tolerante. Cada línea del JSONL resultante
    corresponde a un post extraído, con su fecha asociada si existe.
    
    Args:
        input_dir (str): Ruta del directorio que contiene archivos XML (*.xml)
                         a procesar. Se procesan todos los archivos con extensión .xml
        output_path (str): Ruta del archivo JSONL de salida donde se escribirán
                          los posts extraídos. Se sobrescribe si ya existe
    
    Returns:
        None: La función no retorna valor, pero imprime estadísticas del procesamiento
              y genera el archivo JSONL especificado
    
    """
    pattern = str(Path(input_dir) / "*.xml")
    parser = etree.XMLParser(recover=True, resolve_entities=False)

    total_files = 0
    parsed_ok = 0
    skipped = 0
    total_posts = 0

    with open(output_path, "w", encoding="utf-8") as out:
        for path in glob.glob(pattern):
            total_files += 1
            file_id = Path(path).stem

            try:
                # leer como texto (ignorar errores de codificación raros)
                raw = Path(path).read_text(encoding="utf-8", errors="ignore")
                cleaned = clean_xml_text(raw)

                # parsear con lxml tolerante
                root = etree.fromstring(cleaned.encode("utf-8"), parser=parser)
                if root is None:
                    # lxml no pudo recuperar nada útil
                    skipped += 1
                    print(f"Saltando {path} (irrecuperable tras limpieza)")
                    continue

                dates, posts = extract_texts(root)
                max_len = max(len(dates), len(posts)) if (dates or posts) else 0

                for i in range(max_len):
                    rec = {
                        "source_file": file_id,
                        "index": i,
                        "date": dates[i] if i < len(dates) else "",
                        "post": posts[i] if i < len(posts) else "",
                    }
                    out.write(json.dumps(rec, ensure_ascii=False) + "\n")
                    total_posts += 1

                parsed_ok += 1

            except Exception as e:
                # Si algo explota, seguimos con el siguiente pero reportamos
                skipped += 1
                print(f"Saltando {path} (falló incluso con recover): {e}")

    print(f"Archivos totales: {total_files}")
    print(f"Parseados OK:     {parsed_ok}")
    print(f"Saltados:         {skipped}")
    print(f"Posts escritos:   {total_posts}")
    print(f"Salida JSONL:     {output_path}")

In [5]:

# xml_to_jsonl(PATH_TO_BAC, OUTPUT_BAC_JSONL)

### Dataset 20 Newsgroups

In [6]:
def iter_files(input_dir: str,recursive: bool=False):
    """
    Itera sobre archivos en un directorio que coincidan con un patrón específico.
    
    Busca archivos en el directorio especificado usando globbing patterns.
    Opcionalmente puede buscar de forma recursiva en subdirectorios.
    Los resultados se devuelven ordenados alfabéticamente.
    
    Args:
        input_dir (str): Ruta del directorio donde buscar archivos.
        recursive (bool, optional): Si True, busca recursivamente en subdirectorios.
                                   Si False, solo busca en el directorio raíz.
                                   Defaults to False.
    
    Yields:
        Path: Objeto Path de cada archivo encontrado que coincida con el patrón.
    
    """
    base = Path(input_dir)
    it = base.rglob("*") if recursive else base.glob("*")
    for p in sorted(it):
        if p.is_file():
            yield p

def parse_message(path: Path):
    """
    Parsea un archivo de mensaje extrayendo headers y contenido del cuerpo.
    
    Lee un archivo de mensaje (formato email/newsgroup) y separa las cabeceras
    del cuerpo del mensaje. Maneja cabeceras multi-línea que continúan en
    líneas siguientes con espacios o tabs. Intenta diferentes codificaciones
    para manejar caracteres especiales.
    
    Args:
        path (Path): Ruta del archivo de mensaje a parsear.
    
    Returns:
        dict: Diccionario con las siguientes claves:
            - file_name (str): Nombre del archivo procesado
            - subject (str): Asunto del mensaje (header "Subject")
            - from (str): Remitente del mensaje (header "From") 
            - text (str): Contenido completo del cuerpo del mensaje
    
    """
    try:
        raw = path.read_text(encoding="utf-8", errors="replace")
    except Exception:
        raw = path.read_text(encoding="latin-1", errors="replace")

    lines = raw.splitlines()
    headers = {}
    body_lines = []
    in_headers = True
    current_field = None

    for line in lines:
        if in_headers:
            if line.strip() == "":
                in_headers = False
                continue
            if line[:1] in (" ", "\t") and current_field:
                # continuación de la cabecera previa
                headers[current_field] = (headers[current_field] + " " + line.strip()).strip()
                continue
            if ":" in line:
                name, value = line.split(":", 1)
                current_field = name.strip().lower()
                headers[current_field] = value.strip()
            else:
                # línea sin ":" dentro de la zona de cabeceras → asumimos que empezó el cuerpo
                in_headers = False
                body_lines.append(line)
        else:
            body_lines.append(line)

    return {
        "file_name": path.name,
        "subject": headers.get("subject", ""),
        "from": headers.get("from", ""),
        "text": "\n".join(body_lines).strip()
    }

def folder_to_jsonl(input_dir: str, output_file: str, recursive: bool=False):
    """
    Convierte todos los archivos de mensaje en un directorio a formato JSONL.
    
    Procesa todos los archivos en el directorio especificado, extrae los datos
    de cada mensaje usando parse_message(), y guarda cada mensaje como una línea
    JSON en un archivo JSONL. Crea el directorio de salida si no existe.
    
    Args:
        input_dir (str): Ruta del directorio que contiene los archivos de mensaje.
        output_file (str): Ruta del archivo JSONL de salida donde guardar los datos.
        recursive (bool, optional): Si True, procesa subdirectorios recursivamente.
                                   Si False, solo procesa el directorio raíz.
                                   Defaults to False.
    
    
    """
    out = Path(output_file)
    out.parent.mkdir(parents=True, exist_ok=True)
    with out.open("w", encoding="utf-8") as fout:
        for p in iter_files(input_dir, recursive):
            rec = parse_message(p)
            fout.write(json.dumps(rec, ensure_ascii=False) + "\n")



In [7]:
# folder_to_jsonl(PATH_TO_20NG, OUTPUT_20NG_JSONL, recursive=True)

## Tokenización

Para la tokenización se utilizó la librería `nltk` tanto para la tokenización de oraciones como de palabras. Se seleccionó esta alternativa por encima de usar expresiones regulares para intentar cubrir la mayor cantidad de casos posibles.

In [8]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Se define una expresión regular para identificar tokens que contengan números y reemplazarlos por el token especial `NUM`.


In [9]:
# "¿Es número?" → cualquier token que contenga al menos un dígito
_HAS_DIGIT = re.compile(r".*\d.*", flags=re.ASCII)

In [10]:
def tokenize_sentences_nltk(text: str, language: str = 'english'):
    """
    Tokeniza texto en oraciones usando NLTK.
    
    Utiliza el tokenizador Punkt de NLTK que está entrenado específicamente
    para detectar límites de oraciones en diferentes idiomas.
    
    Args:
        text (str): Texto a tokenizar en oraciones.
        language (str, optional): Idioma para el tokenizador. 
                                Defaults to 'english'.
    
    Returns:
        list[str]: Lista de oraciones encontradas en el texto.
    

    """
    return nltk.sent_tokenize(text, language=language)

In [ ]:
def normalize_text(t: str) -> str:
    """
    Normaliza el texto eliminando caracteres no deseados y convirtiendo a minúsculas.

    Args:
        text (str): El texto a normalizar.

    Returns:
        str: El texto normalizado.
    """
    t = t.strip()
    return re.sub(r"\s+", " ", t)

In [ ]:
def split_sentences(text: str, language: str = 'english'):
    """
    Recibe una cadaena de palabras y devuelve los tokens ntlk
    de cada una de ellas

    Args:
        sent (str): La frase a tokenizar.
        language (str): el idioma de la frase, 
                        para el parametro de ntlk

    Returns:
        (list): la lista de tokens/palabras ntlk
    """
    return tokenize_sentences_nltk(text, language) if text else []

In [ ]:
def load_texts_from_jsonl(in_path: str, text_key: str = "text"):
    """
    Recibe una path y devuleve una lista con todos los valores
    de la llave "text_key" de cada linea

    Args:
        in_path (str): Ubicacion del JSONL a procesar.
        text_key (str): llave a revisar en cada linea del JSONL 

    Returns:
        texts (list): la lista de los contenidos de text_key de
        todos los registros del JSONL
    """
    texts = []
    with open(in_path, "r", encoding="utf-8") as f:
        for line in f:
            if not line.strip():
                continue
            obj = json.loads(line)
            texts.append(obj.get(text_key, ""))
    return texts
        

In [ ]:
def tokenize(sent: str, language: str = 'english'):
    """
    Recibe una cadaena de palabras y devuelve los tokens ntlk
    de cada una de ellas

    Args:
        sent (str): La frase a tokenizar.
        language (str): el idioma de la frase, 
                        para el parametro de ntlk

    Returns:
        (list): la lista de tokens/palabras ntlk
    """
    return nltk.word_tokenize(sent, language=language)

def map_numbers(tok: str) -> str:
    """
    Recibe una cadaena de caracteres y verifica si tiene numeros para
    poder cambiar ese elemento por la cadena de caracteres NUM.

    Args:
        tok (str): El texto a verificar si tiene numeros.

    Returns:
        str: NUM si tiene numeros o la palabra.
    """
    return "NUM" if _HAS_DIGIT.match(tok) else tok

In [15]:
def count_tokens(in_path: str, text_key: str = "text") -> Counter:
    counts = Counter()
    texts = load_texts_from_jsonl(in_path,text_key)
    for text in texts:
        txt = normalize_text(text.lower())
        for sent in split_sentences(txt):
            toks = [map_numbers(t) for t in tokenize(sent)]
            counts.update(toks)
    return counts

In [ ]:
def write_sentences_with_unk(in_path: str, out_path: str, counts: Counter, text_key: str = "text"):
    """
    Separa cada uno de los registros de los datasets en oraciones(sentences).
    Agrega los caracteres de inicio y de terminacion para cada una de las frases.
    Recibe un elemento Counter con el # de apariciones de cada palabra.
    Para cada una de las frases, recorre las palabras y cambia por <UNK>
    las que salgan 1 sola vez en el Counter

    Args:
        in_path (str): El path del JSONL a procesar.
        out_path (str): El path del JSONL a crear.
        counts (Counter): El contador de las palabras.
        text_key(str): la llave dentro de cada registro del JSONL que contiene
                        el texto en lenguaje natural de interes

    Returns:
        No retorna nada, pero escribe el archivo JSONL en el path especificado 
        con las frases ya procesadas.
    """
    texts = load_texts_from_jsonl(in_path, text_key)
    with open(out_path, "w", encoding="utf-8") as out:
        for text in texts:
            txt = normalize_text(text.lower())
            for sent in split_sentences(txt):
                toks = [map_numbers(t) for t in tokenize(sent)]
                toks = [t if counts[t] > 1 else "<UNK>" for t in toks]
                if not toks:
                    continue
                sent_tokens = ["<s>", *toks, "</s>"]
                out.write(json.dumps({"sentence": sent_tokens}, ensure_ascii=False) + "\n")

In [17]:
def preprocess_dataset(input_jsonl: str, output_jsonl: str, text_key: str = "text"):
    counts = count_tokens(input_jsonl, text_key)
    write_sentences_with_unk(input_jsonl, output_jsonl, counts, text_key)

In [18]:
# preprocess_dataset(OUTPUT_20NG_JSONL, "data/processed/20news_tokenized.jsonl")

In [19]:
# preprocess_dataset(OUTPUT_BAC_JSONL, "data/processed/BAC_tokenized.jsonl",text_key="post")

## División de datos de entrenamiento y prueba

Con la ayuda de random se realiza una división 80/20 de los datos para entrenamiento y prueba respectivamente. Se asegura que la división sea reproducible utilizando una semilla fija.

In [ ]:
def split_train_test(input_jsonl, prefix, group_code, out_dir=".", train_ratio=0.8, seed=42):
    """
    Mezcla las frases del archivo JSONL usando la libreria random
    Utiliza la semilla para garantizar qeu se pueda reproducir
    Se crean los archivos JSONL con los datasets de entrenamiento y
    testeo de acuerdo con las proporciones recomendadas y el formato
    de nombramiento

    Args:
        input_jsonl (str): El path del JSONL a procesar.
        prefix (str): Para poder reusar la función para los 2 datasets,
                        se especifica el nombre para el formateo.
        out_dir (str): El path de los JSONL a crear
        train_ratio (float): proporcion de datos para entrenamiento
        seed (int): semilla para el randomizador 

    Returns:
        No retorna nada, pero escribe los JSONL en el path especificado 
        con las frases ya procesadas y randomizadas.
    """

    random.seed(seed)

    # cargar todas las oraciones
    sentences = []
    with open(input_jsonl, "r", encoding="utf-8") as f:
        for line in f:
            if not line.strip():
                continue
            obj = json.loads(line)
            sentences.append(obj)

    # barajar
    random.shuffle(sentences)

    # dividir 80/20
    split_idx = int(len(sentences) * train_ratio)
    train_sents = sentences[:split_idx]
    test_sents = sentences[split_idx:]

    # nombres de salida
    out_train = Path(out_dir) / f"{prefix}_{group_code}_training.jsonl"
    out_test = Path(out_dir) / f"{prefix}_{group_code}_testing.jsonl"

    # guardar
    with open(out_train, "w", encoding="utf-8") as f:
        for s in train_sents:
            f.write(json.dumps(s, ensure_ascii=False) + "\n")

    with open(out_test, "w", encoding="utf-8") as f:
        for s in test_sents:
            f.write(json.dumps(s, ensure_ascii=False) + "\n")

    print(f"Training: {len(train_sents)} → {out_train}")
    print(f"Testing : {len(test_sents)} → {out_test}")



In [21]:


# split_train_test(OUTPUT_20NG_TOKENIZED_JSONL, "20N", GROUP_ID, out_dir="splits")
# split_train_test(OUTPUT_BAC_TOKENIZED_JSONL, "BAC", GROUP_ID, out_dir="splits")


## N- Gramas con suavizado de laplace

Se generó una solución que permite procesar archivos JSONL en batches, de manera que no es necesario cargar todo el archivo en memoria. Esta implementación podría ser más eficientre si en un solo recorrido se generaran los n-gramas de 1 a n, pero por simplicidad se decidió hacer un recorrido por cada n.

In [22]:
def calculate_ngram_probabilities(
    sentences: List[List[str]], 
    n: int = 2, 
    laplace: float = 1.0,
    batch_size: int = 50000
) -> Dict[Tuple[str, ...], float]:
    """
    Calcula las probabilidades de n-gramas usando suavizado de Laplace con procesamiento eficiente en lotes.
    
    Esta función procesa un corpus de oraciones para calcular las probabilidades de n-gramas
    aplicando suavizado de Laplace para manejar n-gramas no vistos. Utiliza procesamiento
    por lotes para optimizar el uso de memoria en corpus grandes.
    
    La probabilidad se calcula como:
    - Para unigramas: P(w) = (count(w) + alpha) / (total_words + alpha * vocab_size)
    - Para n-gramas: P(wn|w1...wn-1) = (count(w1...wn) + alpha) / (count(w1...wn-1) + alpha * vocab_size)

    alpha es el parámetro de suavizado de Laplace, por defecto 1.0.

    Args:
        sentences (List[List[str]]): Lista de oraciones tokenizadas, donde cada oración 
            es una lista de strings representando palabras o tokens.
        n (int, optional): Tamaño del n-grama a calcular. Por defecto 2 (bigramas).
            Debe ser >= 1.
        laplace (float, optional): Factor de suavizado de Laplace (alpha). Por defecto 1.0.
            Valores más altos proporcionan más suavizado.
        batch_size (int, optional): Número de oraciones a procesar por lote para optimizar
            memoria. Por defecto 50000.
    
    Returns:
        Dict[Tuple[str, ...], float]: Diccionario donde las claves son tuplas representando
            n-gramas y los valores son sus probabilidades calculadas con suavizado de Laplace.
        Dict[Tuple[str, ...], float]: Diccionario donde las claves son tuplas representando
            contextos (n-1)-grama y los valores son sus conteos.
    """
    
    # Fase 1: Conteo eficiente
    ngram_counts = Counter()
    context_counts = Counter() if n > 1 else None
    vocab_size = 0
    total_ngrams = 0
    
    # Procesar en lotes para memoria
    for batch_start in range(0, len(sentences), batch_size):
        batch_end = min(batch_start + batch_size, len(sentences))
        batch_sentences = sentences[batch_start:batch_end]
        
        # Set temporal para vocabulario del batch
        batch_vocab = set()
        
        for sent in batch_sentences:
            batch_vocab.update(sent)
            
            # Contar n-gramas
            for i in range(len(sent) - n + 1):
                ngram = tuple(sent[i:i + n])
                ngram_counts[ngram] += 1
                total_ngrams += 1
                
                # Solo calcular contextos si n > 1
                if context_counts is not None:
                    context = tuple(sent[i:i + n - 1])
                    context_counts[context] += 1
        
        # Actualizar tamaño de vocabulario
        vocab_size = len(batch_vocab | set().union(*[
            set(sent) for sent in sentences[:batch_start]
        ])) if batch_start > 0 else len(batch_vocab)

        # Liberar memoria del batch
        del batch_vocab
        gc.collect()
    
    # Calcular vocabulario final una sola vez
    if batch_size < len(sentences):
        all_vocab = set()
        for sent in sentences:
            all_vocab.update(sent)
        vocab_size = len(all_vocab)
        del all_vocab
    
    # Fase 2: Calcular probabilidades
    probabilities = {}
    
    if n == 1:
        # Para unigramas - calcular total una sola vez
        for ngram, count in ngram_counts.items():
            prob = (count + laplace) / (total_ngrams + laplace * vocab_size)
            probabilities[ngram] = prob
    else:
        # Para n-gramas superiores
        for ngram, count in ngram_counts.items():
            context = ngram[:-1]
            context_count = context_counts[context]
            prob = (count + laplace) / (context_count + laplace * vocab_size)
            probabilities[ngram] = prob
    
    context_counts_dict = dict(context_counts) if context_counts else None

    # Limpiar memoria
    del ngram_counts
    gc.collect()

    return probabilities, context_counts_dict

In [23]:
def process_jsonl_for_ngrams(
    jsonl_file_path: str,
    n: int = 2,
    laplace: float = 1.0,
    batch_size: int = 50000
) -> Dict[Tuple[str, ...], float]:
    """
    Lee un archivo JSONL con sentencias y calcula probabilidades de n-gramas.
    
    Args:
        jsonl_file_path: Ruta al archivo JSONL
        n: Tamaño del n-grama (1=unigrama, 2=bigrama, etc.)
        laplace: Factor de suavizado de Laplace
        batch_size: Tamaño del lote para procesamiento en memoria
        max_sentences: Máximo número de sentencias a procesar (None = todas)
        remove_special_tokens: Si remover tokens especiales como <s>, </s>, <UNK>
    
    Returns:
        Diccionario con n-gramas como claves y probabilidades como valores
        Diccionario con contextos (n-1)-grama como claves y sus conteos como valores
    """
    
    sentences = []
    
    print(f"Leyendo archivo JSONL: {jsonl_file_path}")
    
    try:
        with open(jsonl_file_path, 'r', encoding='utf-8') as file:
            for line_num, line in enumerate(file, 1):
                
                line = line.strip()
                if not line:  # Saltar líneas vacías
                    continue
                try:
                    # Parsear JSON
                    data = json.loads(line)
                    
                    # Extraer sentencia
                    if "sentence" not in data:
                        print(f"Advertencia: línea {line_num} no tiene clave 'sentence'")
                        continue
                    
                    sentence = data["sentence"]
                    
                    # Validar que sea una lista
                    if not isinstance(sentence, list):
                        print(f"Advertencia: línea {line_num} - 'sentence' no es una lista")
                        continue
                    
                    # Saltar sentencias vacías
                    if not sentence:
                        continue
                    
                    sentences.append(sentence)
                    
                
                except json.JSONDecodeError as e:
                    print(f"Error parseando JSON en línea {line_num}: {e}")
                    continue
                except Exception as e:
                    print(f"Error procesando línea {line_num}: {e}")
                    continue
    
    except FileNotFoundError:
        raise FileNotFoundError(f"No se encontró el archivo: {jsonl_file_path}")
    except Exception as e:
        raise Exception(f"Error leyendo archivo: {e}")
    
    if not sentences:
        raise ValueError("No se encontraron sentencias válidas en el archivo")
    
    print(f"Total de sentencias cargadas: {len(sentences)}")
    print(f"Calculando {n}-gramas con suavizado de Laplace (α={laplace})...")
    
    # Llamar a la función de n-gramas
    probabilities, context_counts = calculate_ngram_probabilities(
        sentences=sentences,
        n=n,
        laplace=laplace,
        batch_size=batch_size
    )
    
    print(f"Cálculo completado. Total de {n}-gramas únicos: {len(probabilities)}")
    if context_counts:
        print(f"Total de contextos únicos: {len(context_counts)}")
    
    return probabilities, context_counts

In [50]:
unigrams_20n, context_counts_uni = process_jsonl_for_ngrams(
        jsonl_file_path=OUTPUT_20NG_SPLITS_JSONL,
        n=1,
        laplace=1.0,
        batch_size=50000
    )   

Leyendo archivo JSONL: splits/20N_0100_training.jsonl
Total de sentencias cargadas: 230368
Calculando 1-gramas con suavizado de Laplace (α=1.0)...
Cálculo completado. Total de 1-gramas únicos: 72174


In [51]:
bigrams_20n, context_counts_bi = process_jsonl_for_ngrams(
        jsonl_file_path=OUTPUT_20NG_SPLITS_JSONL,
        n=2,
        laplace=1.0,
        batch_size=50000
    )


Leyendo archivo JSONL: splits/20N_0100_training.jsonl
Total de sentencias cargadas: 230368
Calculando 2-gramas con suavizado de Laplace (α=1.0)...
Cálculo completado. Total de 2-gramas únicos: 941694
Total de contextos únicos: 72173


In [52]:
trigrams_20n, context_counts_tri = process_jsonl_for_ngrams(
        jsonl_file_path=OUTPUT_20NG_SPLITS_JSONL,
        n=3,
        laplace=1.0,
        batch_size=50000
    )

Leyendo archivo JSONL: splits/20N_0100_training.jsonl
Total de sentencias cargadas: 230368
Calculando 3-gramas con suavizado de Laplace (α=1.0)...
Cálculo completado. Total de 3-gramas únicos: 2342730
Total de contextos únicos: 939236


In [53]:
with open('ngrams/unigrams_20n.pkl', 'wb') as f:
    pickle.dump(unigrams_20n, f)

with open('ngrams/bigrams_20n.pkl', 'wb') as f:
    pickle.dump(bigrams_20n, f)

with open('ngrams/trigrams_20n.pkl', 'wb') as f:
    pickle.dump(trigrams_20n, f)


with open('ngrams/context_counts_unigrams_20n.pkl', 'wb') as f:
    pickle.dump(context_counts_uni, f)

with open('ngrams/context_counts_bigrams_20n.pkl', 'wb') as f:
    pickle.dump(context_counts_bi, f)

with open('ngrams/context_counts_trigrams_20n.pkl', 'wb') as f:
    pickle.dump(context_counts_tri, f)


In [24]:
unigrams_bac, context_counts_uni_bac = process_jsonl_for_ngrams(
        jsonl_file_path=OUTPUT_BAC_SPLITS_JSONL,
        n=1,
        laplace=1.0,
        batch_size=250000
    )

Leyendo archivo JSONL: splits/BAC_0100_training.jsonl
Total de sentencias cargadas: 7130342
Calculando 1-gramas con suavizado de Laplace (α=1.0)...
Cálculo completado. Total de 1-gramas únicos: 386329


In [25]:
with open('ngrams/unigrams_bac.pkl', 'wb') as f:
    pickle.dump(unigrams_bac, f)

with open('ngrams/context_counts_unigrams_bac.pkl', 'wb') as f:
    pickle.dump(context_counts_uni_bac, f)  # Será None para unigramas

In [26]:

bigrams_bac, context_counts_bi_bac = process_jsonl_for_ngrams(
        jsonl_file_path=OUTPUT_BAC_SPLITS_JSONL,
        n=2,
        laplace=1.0,
        batch_size=250000
    )

Leyendo archivo JSONL: splits/BAC_0100_training.jsonl
Total de sentencias cargadas: 7130342
Calculando 2-gramas con suavizado de Laplace (α=1.0)...
Cálculo completado. Total de 2-gramas únicos: 9861802
Total de contextos únicos: 386328


In [27]:
with open('ngrams/bigrams_bac.pkl', 'wb') as f:
    pickle.dump(bigrams_bac, f)

with open('ngrams/context_counts_bigrams_bac.pkl', 'wb') as f:
    pickle.dump(context_counts_bi_bac, f)

In [28]:

trigrams_bac, context_counts_tri_bac = process_jsonl_for_ngrams(
        jsonl_file_path=OUTPUT_BAC_SPLITS_JSONL,
        n=3,
        laplace=1.0,
        batch_size=250000
    )

Leyendo archivo JSONL: splits/BAC_0100_training.jsonl
Total de sentencias cargadas: 7130342
Calculando 3-gramas con suavizado de Laplace (α=1.0)...
Cálculo completado. Total de 3-gramas únicos: 38191007
Total de contextos únicos: 9842678


In [29]:
with open('ngrams/trigrams_bac.pkl', 'wb') as f:
    pickle.dump(trigrams_bac, f)

with open('ngrams/context_counts_trigrams_bac.pkl', 'wb') as f:
    pickle.dump(context_counts_tri_bac, f)